In [1]:
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_internals import MahalanobisMonitor, GaussianMixtureMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor
from monitors_input import SHINE_monitor
from evaluation import Evaluator

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import utils 

2022-07-11 15:18:45.310629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rsenaferre/anaconda3_new/envs/ANITI_RuntimeMonitoringBenchmark/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-11 15:18:45.310646: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
batch_size = 10
model = "resnet"
# model = "densenet"

layers_ids = [2, 5, 8, 12, 15, 19, 22, 26, 29, 32]
# layers_ids = [9, 19, 29, 40, 50, 60, 69, 79, 89, 98]

id_dataset = "cifar10"

ood_dataset = "svhn"

additional_transform = None#"pixelization"
adversarial_attack = None#"fgsm"

In [3]:
dataset_train = Dataset(id_dataset, "train", model, batch_size=batch_size)
dataset_test = Dataset(id_dataset, "test", model, batch_size=batch_size)
dataset_ood = Dataset(ood_dataset, "test", model, additional_transform, adversarial_attack, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat


In [4]:
feature_extractor = FeatureExtractor(model, id_dataset, layers_ids)

In [6]:
features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(dataset_train)
features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(dataset_test)
features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = feature_extractor.get_features(dataset_ood)

In [7]:
id_accuracy = accuracy_score(lab_test, pred_test)
ood_accuracy = 0
if id_dataset == ood_dataset:
    ood_accuracy = accuracy_score(lab_ood, pred_ood)

print("Accuracy")
print("ID:  ", id_accuracy)
print("OOD: ", ood_accuracy)

Accuracy
ID:   0.9367
OOD:  0


In [21]:

# Converting training dataset (ID) from pytorch to numpy format
num_samples = None

X, y = [],[]

with torch.no_grad():
    for data in tqdm(dataset_train.dataloader):
        X.append(data[0].numpy())
        y.append(data[1].numpy())

X = np.array(X)
y = np.array(y)
X_train = np.reshape(X, (batch_size*X.shape[0], X.shape[3], X.shape[4], X.shape[2]))
y_train = np.reshape(y, batch_size*y.shape[0])

print('np.shape(X), np.shape(y):',np.shape(X_train), np.shape(y_train))

100%|██████████████████████████████████████| 5000/5000 [00:05<00:00, 833.58it/s]


np.shape(X), np.shape(y): (50000, 32, 32, 3) (50000,)


In [8]:
(id_X_train, id_y_train), (id_X_test, id_y_test) = utils.load_data(id_dataset, None)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [22]:
from torch.utils.data import TensorDataset, DataLoader
# verifying if labels are equal (torch2numpy, and data loaded manually)
np.all(y_train == id_y_train)

tensor_x = torch.Tensor(id_X_train) # transform to torch tensor
tensor_y = torch.Tensor(id_y_train)

my_trainset = TensorDataset(tensor_x,tensor_y)
my_trainset.name = id_dataset
my_trainset.split="train"
my_trainset.network=model
my_trainset.batch_size=batch_size
features_train2, logits_train2, softmax_train2, pred_train2, lab_train2 = feature_extractor.get_features(my_trainset)



False

In [19]:
my_dataloader = DataLoader(my_trainset)
# Converting training dataset (ID) from pytorch to numpy format
num_samples = None

X, y = [],[]

with torch.no_grad():
    for data in tqdm(my_dataloader):
        X.append(data[0].numpy())
        y.append(data[1].numpy())

X = np.array(X)
y = np.array(y)
X_train2 = np.reshape(X, (X.shape[0], X.shape[2], X.shape[3], X.shape[4]))
y_train2 = np.reshape(y, y.shape[0])

print('np.shape(X), np.shape(y):',np.shape(X_train2), np.shape(y_train2))

np.all(lab_train == y_train2)

100%|██████████████████████████████████| 50000/50000 [00:02<00:00, 22342.70it/s]


np.shape(X), np.shape(y): (50000, 32, 32, 3) (50000,)


True

In [ ]:
eval_oms = Evaluator("oms", is_novelty=(id_dataset!=ood_dataset))

eval_ood = Evaluator("ood", is_novelty=(id_dataset!=ood_dataset))

eval_oms.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)
eval_ood.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)

precision_star, recall_star, f1_star = eval_oms.get_metrics(eval_ood.y_true[:lab_test.shape[0]].astype(bool), 
                                             eval_ood.y_true[lab_test.shape[0]:].astype(bool))

print("OMS results for perfect OOD detector")
print("Precision ", " Recall ", " F1")
print("{:.4f}".format(precision_star), "    ", "{:.4f}".format(recall_star), " ", "{:.4f}".format(f1_star))

In [ ]:
id_layer_monitored = 3
id_layer_monitored = -1

In [ ]:
monitor_mahalanobis = MahalanobisMonitor(id_dataset, model, id_layer_monitored)
monitor_mahalanobis.fit(features_train[id_layer_monitored], lab_train)

scores_test_mahalanobis = monitor_mahalanobis.predict(features_test[id_layer_monitored], pred_test) 
scores_ood_mahalanobis = monitor_mahalanobis.predict(features_ood[id_layer_monitored], pred_ood)

In [ ]:
from evaluation import get_auroc_ood, get_tnr_frac_tpr_ood

monitor_mahalanobis = MahalanobisMonitor(id_dataset, model, id_layer_monitored, is_tied=True)
monitor_mahalanobis.fit(features_train[id_layer_monitored], lab_train)

scores_test_mahalanobis = monitor_mahalanobis.predict(features_test[id_layer_monitored], pred_test) 
scores_ood_mahalanobis = monitor_mahalanobis.predict(features_ood[id_layer_monitored], pred_ood)

a = get_auroc_ood(scores_test_mahalanobis, scores_ood_mahalanobis)


b = get_tnr_frac_tpr_ood(scores_test_mahalanobis, scores_ood_mahalanobis)
a, b

In [ ]:
plt.hist(scores_test_mahalanobis)
plt.hist(scores_ood_mahalanobis, alpha=0.5)
plt.show()

In [ ]:
# ID dataset
from tqdm import tqdm
import torch
import utils
num_samples = None

X, y = [],[]

with torch.no_grad():
    for data in tqdm(dataset_train.dataloader):
        X.append(data[0].numpy())
        y.append(data[1].numpy())

X = np.array(X)
y = np.array(y)
X = np.reshape(X, (batch_size*X.shape[0], X.shape[3], X.shape[4], X.shape[2]))
y = np.reshape(y, batch_size*y.shape[0])

print('np.shape(X), np.shape(y):',np.shape(X), np.shape(y))

In [ ]:
#building monitor with ID data
monitor_shine = SHINE_monitor(id_dataset)
monitor_shine.fit_by_class_parallel(X, y)

print('number of monitors',len(monitor_shine.arr_density))

In [ ]:
#loading test ID
X, y = [],[]

for data in tqdm(dataset_test.dataloader):
    X.append(data[0].numpy())
    y.append(data[1].numpy())
    
X = np.array(X)
y = np.array(y)
print(np.shape(X), np.shape(y), np.shape(pred_test))

X = np.reshape(X, (batch_size*X.shape[0], X.shape[3], X.shape[4], X.shape[2]))
y = np.reshape(y, batch_size*y.shape[0])

print(np.shape(X), np.shape(y), np.shape(pred_test))

In [ ]:
#testing ID
arr_score_ID = {}
arr_score_OOD = {}

# looping through all monitored classes
for monitored_class in range(10):
    for x, pred, label in tqdm(zip(X, pred_test, y)):
        pdf = monitor_shine.predict_parallel(np.array([x]), pred)

        if monitored_class == label:
            try:
                arr_score_ID[monitored_class].append(pdf)
            except:
                arr_score_ID.update({monitored_class: [pdf]})
        else:
            try:
                arr_score_OOD[monitored_class].append(pdf)
            except:
                arr_score_OOD.update({monitored_class: [pdf]})


In [ ]:
#testing OOD
X_ood, y_ood = [],[]

for monitored_class in range(10):
    for data in tqdm(dataset_ood.dataloader):
        X = data[0].numpy()
        X_ood = np.reshape(X, (X.shape[0], X.shape[2], X.shape[3], X.shape[1]))
        #y_ood = data[1].numpy()
        #print(np.shape(X_ood), np.shape(y_ood))

        for x, pred in zip(X_ood, pred_ood):
            pdf = monitor_shine.predict_parallel(np.array([x]), pred)
            
            try:
                arr_score_OOD[monitored_class].append(pdf)
            except:
                arr_score_OOD.update({monitored_class: [pdf]})
            
#arr_score_ID = np.array(arr_score_ID)
#arr_score_OOD = np.array(arr_score_OOD)


In [ ]:
from evaluation import get_auroc_ood, get_auroc_oms, get_tnr_frac_tpr_ood, get_tnr_frac_tpr_oms

auroc, tnr_tpr = 0, 0
for c in range(10): 
    # evaluating shine
    print(c,np.shape(arr_score_ID[c]), np.shape(arr_score_OOD[c]))
    a = get_auroc_ood(np.array(arr_score_ID[c]), np.array(arr_score_OOD[c]))
    print('AUROC class',c,a)
    auroc+=a

    b = get_tnr_frac_tpr_ood(np.array(arr_score_ID[c]), np.array(arr_score_OOD[c]))
    print('TNR/TPR class',c,b)
    tnr_tpr+=b
    
    print('\n')
    
print('avg', auroc/10,  tnr_tpr/10)

In [ ]:
# testing score as a sum of class scores
0.38845303664720343
0.07098955132145052

# testing score as based on a predicted class but together with the other classes 
0.42577552627535337
0.3809157959434542

# testing per monitor (oms)
0.4292515820137766
0.16028501781361335

In [ ]:
import evaluation
#metrics

print('SHINE roc curve')
evaluation.plot_roc_curve(arr_score_ID, arr_score_OOD)

print('SHINE other metrics')

In [ ]:
precision_ood_shine, recall_ood_shine, f1_ood_shine = eval_ood.get_metrics(arr_is_ood_1.astype(bool), 
                                                                            arr_is_ood_2.astype(bool))
#precision_oms_shine, recall_oms_shine, f1_oms_shine = eval_oms.get_metrics(scores_test_shine, 
#                                                                        scores_ood_shine)

print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_shine), "    ", "{:.4f}".format(recall_ood_shine), 
      " ", "{:.4f}".format(f1_ood_shine))

In [ ]:
# mean
Precision   Recall   F1
OOD:    0.6858      0.8316   0.7517

#75
Precision   Recall   F1
OOD:    0.6855      0.8327   0.7520
    
#25
Precision   Recall   F1
OOD:    0.6858      0.8316   0.7517

In [ ]:
# looping through all monitored classes
monitored_classes = [i for i in range(10)]

# scores for each monitored class
arr_pred_test_per_class, arr_pred_ood_per_class = {}, {}

for monitored_class in monitored_classes:
    # we will then monitor the predicted class from the ML model
    predicted_ML = [monitored_class]*len(ood_X_test)
    arr_pred_test, arr_pred_ood = [], []
    
    for x, y_true, pred in zip(X_test, y_test, predicted_ML):
        if y_true == monitored_class:
            is_ood, shine_score = monitor_shine.predict(np.array([x]), pred)    
            arr_pred_test.append(is_ood)
            
    arr_pred_test_per_class.update({monitored_class: np.array(arr_pred_test)})

    for x, pred in zip(ood_X_test, predicted_ML):
        is_ood, shine_score = monitor_shine.predict(np.array([x]), pred)    
        arr_pred_ood.append(is_ood)
        
    arr_pred_ood_per_class.update({monitored_class: np.array(arr_pred_ood)})

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, matthews_corrcoef

# converting the labels from test dataset to the format 1 and 0
y_true = utils.convert_ood_labels(ood_y_test)
# printing balanced accuracy
acc = balanced_accuracy_score(y_true, arr_pred_ood)
print(acc)

In [ ]:
#eval_oms = Evaluator("oms", is_novelty=(id_dataset!=ood_dataset))
eval_ood = Evaluator("ood", is_novelty=(id_dataset!=ood_dataset))

for monitored_class in monitored_classes:
    arr_pred_test, arr_pred_ood = arr_pred_test_per_class[monitored_class], arr_pred_ood_per_class[monitored_class]
    #eval_oms.fit_ground_truth(y_test, ood_y_test, arr_pred_test, arr_pred_ood)
    eval_ood.fit_ground_truth(y_test[y_test==monitored_class], ood_y_test, arr_pred_test, arr_pred_ood)

    precision_ood_shine, recall_ood_shine, f1_ood_shine = eval_ood.get_metrics(arr_pred_test.astype(bool), 
                                                                            arr_pred_ood.astype(bool))
    #precision_oms_shine, recall_oms_shine, f1_oms_shine = eval_oms.get_metrics(scores_test_shine, 
    #                                                                        scores_ood_shine)

    print("Shine when monitoring the class", monitored_class)
    print("       ", "Precision ", " Recall ", " F1")
    print("OOD:   ", "{:.4f}".format(precision_ood_shine), "    ", "{:.4f}".format(recall_ood_shine), 
          " ", "{:.4f}".format(f1_ood_shine))
    #print("OMS:   ", "{:.4f}".format(precision_oms_shine), "    ", "{:.4f}".format(recall_oms_shine), 
    #      " ", "{:.4f}".format(f1_oms_shine))

In [ ]:
monitor_oob = OutsideTheBoxMonitor(n_clusters=10)
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

scores_oob_test = monitor_oob.predict(features_test[id_layer_monitored], pred_test)
scores_oob_ood = monitor_oob.predict(features_ood[id_layer_monitored], pred_ood)

precision_ood_oob, recall_ood_oob, f1_ood_oob = eval_ood.get_metrics(scores_oob_test.astype(bool), scores_oob_ood.astype(bool))
precision_oms_oob, recall_oms_oob, f1_oms_oob = eval_oms.get_metrics(scores_oob_test.astype(bool), scores_oob_ood.astype(bool))

print("Outside the box")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_oob), "    ", "{:.4f}".format(recall_ood_oob), 
      " ", "{:.4f}".format(f1_ood_oob))
print("OMS:   ", "{:.4f}".format(precision_oms_oob), "    ", "{:.4f}".format(recall_oms_oob), 
      " ", "{:.4f}".format(f1_oms_oob))

In [ ]:
monitor_msp = MaxSoftmaxProbabilityMonitor()
monitor_msp.fit()

scores_test_msp = monitor_msp.predict(softmax_test) 
scores_ood_msp = monitor_msp.predict(softmax_ood) 

precision_ood_msp, recall_ood_msp, f1_ood_msp = eval_ood.get_metrics(scores_test_msp.astype(bool), scores_ood_msp.astype(bool))
precision_oms_msp, recall_oms_msp, f1_oms_msp = eval_oms.get_metrics(scores_test_msp.astype(bool), scores_ood_msp.astype(bool))

print("Max Softmax Probability")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_msp), "    ", "{:.4f}".format(recall_ood_msp), 
      " ", "{:.4f}".format(f1_ood_msp))
print("OMS:   ", "{:.4f}".format(precision_oms_msp), "    ", "{:.4f}".format(recall_oms_msp), 
      " ", "{:.4f}".format(f1_oms_msp))

In [ ]:
monitor_react = ReActMonitor(quantile_value=0.99, mode="msp")
monitor_react.fit(feature_extractor, features_train[-1])

scores_test_react = monitor_react.predict(features_test[-1])
scores_ood_react = monitor_react.predict(features_ood[-1])

precision_ood_react_msp, recall_ood_react_msp, f1_ood_react_msp = eval_ood.get_metrics(scores_test_react, 
                                                                                       scores_ood_react)
precision_oms_react_msp, recall_oms_react_msp, f1_oms_react_msp = eval_oms.get_metrics(scores_test_react, 
                                                                                       scores_ood_react)

print("ReAct MSP")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_react_msp), "    ", "{:.4f}".format(recall_ood_react_msp), 
      " ", "{:.4f}".format(f1_ood_react_msp))
print("OMS:   ", "{:.4f}".format(precision_oms_react_msp), "    ", "{:.4f}".format(recall_oms_react_msp), 
      " ", "{:.4f}".format(f1_oms_react_msp))

In [ ]:
monitor_maxlogits = MaxLogitMonitor()
monitor_maxlogits.fit()

scores_test_maxlogits = monitor_maxlogits.predict(logits_test) 
scores_ood_maxlogits = monitor_maxlogits.predict(logits_ood) 

precision_ood, recall_ood, f1_ood = eval_ood.get_metrics(scores_test_maxlogits, scores_ood_maxlogits)
precision_oms, recall_oms, f1_oms = eval_oms.get_metrics(scores_test_maxlogits, scores_ood_maxlogits)

print("Max Logit")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood), "    ", "{:.4f}".format(recall_ood), " ", "{:.4f}".format(f1_ood))
print("OMS:   ", "{:.4f}".format(precision_oms), "    ", "{:.4f}".format(recall_oms), " ", "{:.4f}".format(f1_oms))

In [ ]:
T = 1

monitor_energy = EnergyMonitor(temperature=T)
monitor_energy.fit()

scores_test_energy = monitor_energy.predict(logits_test)
scores_ood_energy = monitor_energy.predict(logits_ood)

precision_ood_energy, recall_ood_energy, f1_ood_energy = eval_ood.get_metrics(scores_test_energy, scores_ood_energy)
precision_oms_energy, recall_oms_energy, f1_oms_energy = eval_oms.get_metrics(scores_test_energy, scores_ood_energy)

print("Energy")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_energy), "    ", "{:.4f}".format(recall_ood_energy), 
      " ", "{:.4f}".format(f1_ood_energy))
print("OMS:   ", "{:.4f}".format(precision_oms_energy), "    ", "{:.4f}".format(recall_oms_energy), 
      " ", "{:.4f}".format(f1_oms_energy))

In [ ]:
monitor_react = ReActMonitor(quantile_value=0.99)
monitor_react.fit(feature_extractor, features_train[-1])

scores_test_react = monitor_react.predict(features_test[-1])
scores_ood_react = monitor_react.predict(features_ood[-1])

precision_ood_react_ene, recall_ood_react_ene, f1_ood_react_ene = eval_ood.get_metrics(scores_test_react, 
                                                                                       scores_ood_react)
precision_oms_react_ene, recall_oms_react_ene, f1_oms_react_ene = eval_oms.get_metrics(scores_test_react, 
                                                                                       scores_ood_react)

print("ReAct Energy")
print("       ", "Precision ", " Recall ", " F1")
print("OOD:   ", "{:.4f}".format(precision_ood_react_ene), "    ", "{:.4f}".format(recall_ood_react_ene), 
      " ", "{:.4f}".format(f1_ood_react_ene))
print("OMS:   ", "{:.4f}".format(precision_oms_react_ene), "    ", "{:.4f}".format(recall_oms_react_ene), 
      " ", "{:.4f}".format(f1_oms_react_ene))